## BIOINFORMATICS THESIS: MULTIMODAL NEURAL NETWORK

# CELL LINE: K562

In [1]:
import pandas as pd 
import numpy as np
import os

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict, OrderedDict
import pickle

import sqlite3
from sqlalchemy import create_engine

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# create a database to store optuna studies with sqlite backend
#engine = create_engine('sqlite:///BIOINF_optuna_tuning.db')

In [3]:
from BIOINF_tesi.data_pipe import CELL_LINES, TASKS

In [4]:
cell_line = CELL_LINES[5]
cell_line

'K562'

---

In [5]:
from BIOINF_tesi.data_pipe import Load_Create_Task
from BIOINF_tesi.data_pipe import Build_DataLoader_Pipeline

In [6]:
from BIOINF_tesi.models import FFNN, CNN, CNN_LSTM
from BIOINF_tesi.models.utils import fit, Param_Search, Kfold_CV

In [7]:
with open ('results_dict.pickle', 'rb') as fin:
    results_dict = pickle.load(fin)
    results_dict = defaultdict(lambda: defaultdict(dict), results_dict)

## 1) ACTIVE ENHANCERS vs INACTIVE ENHANCERS

In [8]:
task = TASKS[0]
task

'active_E_vs_inactive_E'

In [9]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


### 1. FFNN

In [10]:
model=FFNN

In [11]:
# IMBALANCED
type_augm_genfeatures = 'smote'

kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                rebalancing=True,
                type_augm_genfeatures=type_augm_genfeatures,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:370: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-11 22:31:20,315] A new study created in RDB with name: K562_active_E_vs_inactive_E_FFNN_smote_1


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-11 22:37:24,109] Trial 0 finished with value: 0.0865316901408451 and parameters: {'n_layers': 3, 'n_units_l0': 128, 'dropout_l0': 0.3, 'n_units_l1': 16, 'dropout_l1': 0.4, 'n_units_l2': 32, 'dropout_l2': 0.5, 'optimizer': 'RMSprop', 'lr': 0.08989603541802824, 'weight_decay': 0.04817876843912466}. Best is trial 0 with value: 0.0865316901408451.


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-11 23:32:57,701] Trial 2 finished with value: 0.15907638075008804 and parameters: {'n_layers': 3, 'n_units_l0': 64, 'dropout_l0': 0.2, 'n_units_l1': 64, 'dropout_l1': 0.4, 'n_units_l2': 32, 'dropout_l2': 0.0, 'optimizer': 'RMSprop', 'lr': 3.891108536290205e-05, 'weight_decay': 0.0009862765076491997}. Best is trial 2 with value: 0.15907638075008804.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.15907638075008804
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.4
    dropout_l2: 0.0
    lr: 3.891108536290205e-05
    n_layers: 3
    n_units_l0: 64
    n_units_l1: 64
    n_units_l2: 32
    optimizer: RMSprop
    weight_decay: 0.0009862765076491997

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.16145775108191054



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.15612


In [12]:
results_dict[cell_line][task][f'{model.__name__}_{type_augm_genfeatures}'] = kf_CV.scores_dict

In [13]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

In [14]:
# IMBALANCED
type_augm_genfeatures = 'double'

kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                rebalancing=True,
                type_augm_genfeatures=type_augm_genfeatures,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:370: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-11 23:38:27,541] A new study created in RDB with name: K562_active_E_vs_inactive_E_FFNN_double_1


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-11 23:42:06,167] Trial 0 finished with value: 0.16197792747002746 and parameters: {'n_layers': 1, 'n_units_l0': 128, 'dropout_l0': 0.2, 'optimizer': 'RMSprop', 'lr': 0.00012028090535809403, 'weight_decay': 0.01620646565837296}. Best is trial 0 with value: 0.16197792747002746.


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-12 00:34:03,569] Trial 2 finished with value: 0.1389375657777646 and parameters: {'n_layers': 2, 'n_units_l0': 64, 'dropout_l0': 0.3, 'n_units_l1': 16, 'dropout_l1': 0.4, 'optimizer': 'RMSprop', 'lr': 1.76863358673458e-05, 'weight_decay': 0.06750312396948567}. Best is trial 1 with value: 0.1828160480803533.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.1828160480803533
  Params: 
    dropout_l0: 0.3
    dropout_l1: 0.2
    dropout_l2: 0.5
    lr: 0.01373981558615264
    n_layers: 3
    n_units_l0: 64
    n_units_l1: 128
    n_units_l2: 64
    optimizer: RMSprop
    weight_decay: 0.0001511965897094776

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.10636966493441681



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.14717


In [15]:
results_dict[cell_line][task][f'{model.__name__}_{type_augm_genfeatures}'] = kf_CV.scores_dict

In [16]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [10]:
model=CNN

In [11]:
# IMBALANCED
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=True,
                rebalancing=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:370: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-12 08:46:49,156] A new study created in RDB with name: K562_active_E_vs_inactive_E_CNN_1


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-12 11:18:15,866] Trial 2 finished with value: 0.08808098591549295 and parameters: {'n_layers': 4, 'out_channels_l0': 64, 'kernel_size_l0': 15, 'dropout_l0': 0.3, 'out_channels_l1': 64, 'kernel_size_l1': 11, 'dropout_l1': 0.5, 'out_channels_l2': 96, 'kernel_size_l2': 5, 'dropout_l2': 0, 'out_channels_l3': 128, 'kernel_size_l3': 15, 'dropout_l3': 0.4, 'optimizer': 'Adam', 'lr': 0.0005607959826877003, 'weight_decay': 0.0007966659424304436}. Best is trial 0 with value: 0.09885490499419419.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.09885490499419419
  Params: 
    dropout_l0: 0
    kernel_size_l0: 5
    lr: 0.0004624916299522377
    n_layers: 1
    optimizer: RMSprop
    out_channels_l0: 16
    weight_decay: 0.03887450496714792

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.08617179741807351



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.09567


In [12]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [13]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 2) ACTIVE PROMOTERS vs INACTIVE PROMOTERS

In [14]:
task = TASKS[1]
task

'active_P_vs_inactive_P'

In [15]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---

### 1. FFNN

In [19]:
model=FFNN

In [20]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                rebalancing=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:370: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-12 00:50:28,665] A new study created in RDB with name: K562_active_P_vs_inactive_P_FFNN_1



===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-12 02:05:56,691] Trial 2 finished with value: 0.1312596339837236 and parameters: {'n_layers': 3, 'n_units_l0': 64, 'dropout_l0': 0.3, 'n_units_l1': 128, 'dropout_l1': 0.2, 'n_units_l2': 16, 'dropout_l2': 0.0, 'optimizer': 'RMSprop', 'lr': 1.028921694274926e-05, 'weight_decay': 0.022453369914131507}. Best is trial 2 with value: 0.1312596339837236.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.1312596339837236
  Params: 
    dropout_l0: 0.3
    dropout_l1: 0.2
    dropout_l2: 0.0
    lr: 1.028921694274926e-05
    n_layers: 3
    n_units_l0: 64
    n_units_l1: 128
    n_units_l2: 16
    optimizer: RMSprop
    weight_decay: 0.022453369914131507

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.16695421647225367



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.28938


In [21]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [22]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [16]:
model=CNN

In [17]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=True,
                rebalancing=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:370: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-12 11:32:46,679] A new study created in RDB with name: K562_active_P_vs_inactive_P_CNN_1


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-12 11:55:09,819] Trial 0 finished with value: 0.1848284357517103 and parameters: {'n_layers': 1, 'out_channels_l0': 64, 'kernel_size_l0': 5, 'dropout_l0': 0.4, 'optimizer': 'RMSprop', 'lr': 0.005701566059025464, 'weight_decay': 0.00683843177528163}. Best is trial 0 with value: 0.1848284357517103.


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-12 15:37:58,730] Trial 2 finished with value: 0.2394728006625121 and parameters: {'n_layers': 1, 'out_channels_l0': 32, 'kernel_size_l0': 15, 'dropout_l0': 0.4, 'optimizer': 'Adam', 'lr': 0.0033025339374387905, 'weight_decay': 0.0033168947572443184}. Best is trial 0 with value: 0.24844796603333708.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.24844796603333708
  Params: 
    dropout_l0: 0.3
    kernel_size_l0: 11
    lr: 0.0031527649209093034
    n_layers: 1
    optimizer: Adam
    out_channels_l0: 64
    weight_decay: 0.00044860548918871013

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.24195604181140212



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.22602


In [18]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [19]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 3) ACTIVE ENHANCERS vs ACTIVE PROMOTERS

In [20]:
task = TASKS[2]
task

'active_E_vs_active_P'

In [21]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### 1. FFNN

In [25]:
model=FFNN

In [26]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                rebalancing=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:370: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-12 02:13:11,723] A new study created in RDB with name: K562_active_E_vs_active_P_FFNN_1



===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-12 02:25:45,982] Trial 2 finished with value: 0.7248912728868855 and parameters: {'n_layers': 2, 'n_units_l0': 32, 'dropout_l0': 0.3, 'n_units_l1': 16, 'dropout_l1': 0.3, 'optimizer': 'RMSprop', 'lr': 2.138747810080283e-05, 'weight_decay': 0.0005739681205396538}. Best is trial 1 with value: 0.7582116810297906.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7582116810297906
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.3
    dropout_l2: 0.0
    dropout_l3: 0.0
    lr: 0.0017232667779949508
    n_layers: 4
    n_units_l0: 256
    n_units_l1: 64
    n_units_l2: 32
    n_units_l3: 4
    optimizer: Nadam
    weight_decay: 0.00023304579785823218

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.8048812285939689



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.79017


In [27]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [28]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [22]:
model=CNN

In [23]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=True,
                rebalancing=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:370: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-12 16:22:56,460] A new study created in RDB with name: K562_active_E_vs_active_P_CNN_1


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-12 17:27:15,373] Trial 2 finished with value: 0.5999128790016992 and parameters: {'n_layers': 4, 'out_channels_l0': 16, 'kernel_size_l0': 11, 'dropout_l0': 0, 'out_channels_l1': 96, 'kernel_size_l1': 5, 'dropout_l1': 0.5, 'out_channels_l2': 64, 'kernel_size_l2': 11, 'dropout_l2': 0.5, 'out_channels_l3': 512, 'kernel_size_l3': 11, 'dropout_l3': 0, 'optimizer': 'RMSprop', 'lr': 1.4969574802333853e-05, 'weight_decay': 0.00018815117490107107}. Best is trial 1 with value: 0.7512711775914562.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7512711775914562
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.4
    dropout_l2: 0
    dropout_l3: 0
    kernel_size_l0: 5
    kernel_size_l1: 11
    kernel_size_l2: 5
    kernel_size_l3: 15
    lr: 0.004373386569316851
    n_layers: 4
    optimizer: Adam
    out_channels_l0: 16
    out_channels_l1: 96
    out_channels_l2: 128
    out_channels_l3: 256
    weight_decay: 0.0010154843912017033

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.7295328074024247



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.57911


In [24]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [25]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 4) INACTIVE ENHANCERS vs INACTIVE PROMOTERS

In [26]:
task = TASKS[3]
task

'inactive_E_vs_inactive_P'

In [27]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### 1. FFNN

In [31]:
model=FFNN

In [32]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                rebalancing=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:370: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-12 02:28:56,978] A new study created in RDB with name: K562_inactive_E_vs_inactive_P_FFNN_1



===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-12 04:55:58,045] Trial 2 finished with value: 0.3962824389014866 and parameters: {'n_layers': 4, 'n_units_l0': 128, 'dropout_l0': 0.3, 'n_units_l1': 32, 'dropout_l1': 0.4, 'n_units_l2': 4, 'dropout_l2': 0.0, 'n_units_l3': 4, 'dropout_l3': 0.4, 'optimizer': 'Adam', 'lr': 0.0028724165311694765, 'weight_decay': 0.0036001128094182147}. Best is trial 0 with value: 0.5659568773465478.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5659568773465478
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.3
    lr: 0.0001670063847371949
    n_layers: 2
    n_units_l0: 256
    n_units_l1: 32
    optimizer: Nadam
    weight_decay: 0.004064442389538894

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.5766967412737221



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.57036


In [33]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [34]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [28]:
model=CNN

In [29]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=True,
                rebalancing=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:370: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-12 17:35:27,529] A new study created in RDB with name: K562_inactive_E_vs_inactive_P_CNN_1


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-12 18:06:11,730] Trial 0 finished with value: 0.5316655380596873 and parameters: {'n_layers': 3, 'out_channels_l0': 32, 'kernel_size_l0': 15, 'dropout_l0': 0, 'out_channels_l1': 64, 'kernel_size_l1': 15, 'dropout_l1': 0, 'out_channels_l2': 96, 'kernel_size_l2': 15, 'dropout_l2': 0.4, 'optimizer': 'Nadam', 'lr': 0.014714848469556502, 'weight_decay': 0.02259534978909389}. Best is trial 0 with value: 0.5316655380596873.


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-13 02:18:29,388] Trial 2 finished with value: 0.5613009655669942 and parameters: {'n_layers': 1, 'out_channels_l0': 32, 'kernel_size_l0': 15, 'dropout_l0': 0.2, 'optimizer': 'RMSprop', 'lr': 0.0006138929936005074, 'weight_decay': 0.0011407636912847225}. Best is trial 1 with value: 0.5728102802030217.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5728102802030217
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.5
    dropout_l2: 0.4
    dropout_l3: 0.5
    kernel_size_l0: 5
    kernel_size_l1: 5
    kernel_size_l2: 11
    kernel_size_l3: 15
    lr: 0.00016607975331972227
    n_layers: 4
    optimizer: Nadam
    out_channels_l0: 64
    out_channels_l1: 32
    out_channels_l2: 64
    out_channels_l3: 512
    weight_decay: 0.008277611566868029

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.4631413520363631



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.53238


In [30]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [31]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 5) ACTIVE ENHANCERS + ACTIVE PROMOTERS vs INACTIVE REST

In [32]:
task = TASKS[4]
task

'active_EP_vs_inactive_rest'

In [33]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### 1. FFNN

In [37]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                rebalancing=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:370: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-12 05:17:29,172] A new study created in RDB with name: K562_active_EP_vs_inactive_rest_FFNN_1



===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-12 07:46:00,367] Trial 2 finished with value: 0.11162087912087906 and parameters: {'n_layers': 3, 'n_units_l0': 32, 'dropout_l0': 0.2, 'n_units_l1': 16, 'dropout_l1': 0.0, 'n_units_l2': 64, 'dropout_l2': 0.5, 'optimizer': 'Adam', 'lr': 0.01963204881164063, 'weight_decay': 0.023255687400315184}. Best is trial 1 with value: 0.23629712315549586.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.23629712315549586
  Params: 
    dropout_l0: 0.3
    lr: 0.04779724877395587
    n_layers: 1
    n_units_l0: 64
    optimizer: Adam
    weight_decay: 0.0004217457209240847

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.23775325649207518



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.25622


In [38]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [39]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [34]:
model=CNN

In [ ]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=True,
                rebalancing=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                hp_model_path = f'{cell_line}_{task}_{model.__name__}_HP',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:370: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-13 03:19:13,321] A new study created in RDB with name: K562_active_EP_vs_inactive_rest_CNN_1


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-13 03:43:28,601] Trial 0 finished with value: 0.16530431024701364 and parameters: {'n_layers': 3, 'out_channels_l0': 64, 'kernel_size_l0': 5, 'dropout_l0': 0.2, 'out_channels_l1': 32, 'kernel_size_l1': 15, 'dropout_l1': 0.4, 'out_channels_l2': 64, 'kernel_size_l2': 15, 'dropout_l2': 0.5, 'optimizer': 'Nadam', 'lr': 0.00018100872559011593, 'weight_decay': 0.00038863492714039436}. Best is trial 0 with value: 0.16530431024701364.


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:370: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-13 10:15:19,008] A new study created in RDB with name: K562_active_EP_vs_inactive_rest_CNN_1_2_3


EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.14636821543639758


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5


In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)